In [246]:
import pandas as pd   
import json
import requests
from pandas import json_normalize

In [247]:
df_raw = pd.read_json(path_or_buf='./data-extraction/twitter_monkeypox.jsonl', lines=True)
df_raw.head()


,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,snscrape.modules.twitter.Tweet,https://twitter.com/IMSS_HGO/status/1628385619...,2023-02-22 13:26:00+00:00,🧐 Con medidas preventivas sencillas se puede d...,🧐 Con medidas preventivas sencillas se puede d...,1628385619326955521,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,None,None,None,"[MonkeyPox, ViruelaDelMono]",None,None,4.0,None
1,snscrape.modules.twitter.Tweet,https://twitter.com/OMSCentrafrique/status/162...,2023-02-22 11:43:51+00:00,L'@OMSCentrafrique appuie la formation de 4️⃣8...,L'@OMSCentrafrique appuie la formation de 4️⃣8...,1628359913050189825,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,4,4,...,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,"[santé, Monkeypox, choléra, RCA, Monkeypox]",None,None,184.0,None
2,snscrape.modules.twitter.Tweet,https://twitter.com/Univadis/status/1628356434...,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#Monkeypox remains global health emergency #Me...,1628356434218868738,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,None,None,None,"[Monkeypox, MedTwitter]",None,None,16.0,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/sanidadgob/status/16283551...,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,1628355168587726851,"{'_type': 'snscrape.modules.twitter.User', 'us...",2,2,2,...,NaN,None,None,None,None,"[ViruelaDelMono, Monkeypox]",None,None,2613.0,None
4,snscrape.modules.twitter.Tweet,https://twitter.com/redaccionmedica/status/162...,2023-02-22 10:37:42+00:00,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,1628343266893602817,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,2,...,NaN,None,None,None,None,[monkeypox],None,{'_type': 'snscrape.modules.twitter.SummaryCar...,545.0,None


In [248]:
df=df_raw.copy()

In [249]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df['username']=df.url.apply(lambda x: x.split('/')[3])
df['displayname']=df.user.apply(lambda x:x['displayname'])
df['followers_count']=df.user.apply(lambda x:x['followersCount'])
df['friends_count']=df.user.apply(lambda x:x['friendsCount'])
df['location']=df.user.apply(lambda x:x['location'])
df['media_count']=df.user.apply(lambda x:x['mediaCount'])
df = df.astype({"viewCount": int},errors='ignore')

In [250]:
df=df.drop('_type',axis=1)
df=df.drop('url',axis=1)
df=df.drop('user',axis=1)
df=df.drop('id',axis=1)
df=df.drop('cashtags',axis=1)
df=df.drop('retweetedTweet',axis=1)
df=df.drop('coordinates',axis=1)
df=df.drop('place',axis=1)
df=df.drop('sourceUrl',axis=1)
df=df.drop('vibe',axis=1)
df=df.drop('mentionedUsers',axis=1)
df=df.drop('inReplyToTweetId',axis=1)
df=df.drop('inReplyToUser',axis=1)
df=df.drop('links',axis=1)
df=df.drop('media',axis=1)
df=df.drop('source',axis=1)
df=df.drop('sourceLabel',axis=1)
df=df.drop('card',axis=1)
df=df.drop('quotedTweet',axis=1)

In [251]:
df=df.rename(columns={"rawContent": "raw_content", "renderedContent": "rendered_content",
            "replyCount": "reply_count", "retweetCount": "retweet_count",
             "likeCount": "like_count", "quoteCount": "quote_count",
             "viewCount": "view_count", "conversationId": "conversation_id"
             
                     })

In [252]:
df=df[['username', 'displayname', 'year', 'month', 'day', 'lang', 'location',
       'raw_content', 'rendered_content', 'reply_count', 'retweet_count',
       'like_count', 'quote_count', 'view_count',
       'followers_count', 'friends_count', 'media_count','date','conversation_id','hashtags']]

In [253]:
df.head()

,username,displayname,year,month,day,lang,location,raw_content,rendered_content,reply_count,retweet_count,like_count,quote_count,view_count,followers_count,friends_count,media_count,date,conversation_id,hashtags
0,IMSS_HGO,IMSS Hidalgo,2023,2,22,es,"Hidalgo, Pachuca de Soto",🧐 Con medidas preventivas sencillas se puede d...,🧐 Con medidas preventivas sencillas se puede d...,0,0,0,0,4.0,3164,269,3326,2023-02-22 13:26:00+00:00,1628385619326955521,"[MonkeyPox, ViruelaDelMono]"
1,OMSCentrafrique,OMS Centrafrique,2023,2,22,fr,"Bangui, RCA",L'@OMSCentrafrique appuie la formation de 4️⃣8...,L'@OMSCentrafrique appuie la formation de 4️⃣8...,1,4,4,0,184.0,6371,224,486,2023-02-22 11:43:51+00:00,1628359913050189825,"[santé, Monkeypox, choléra, RCA, Monkeypox]"
2,Univadis,Univadis,2023,2,22,en,,#Monkeypox remains global health emergency #Me...,#Monkeypox remains global health emergency #Me...,0,0,0,0,16.0,626,997,3202,2023-02-22 11:30:01+00:00,1628356434218868738,"[Monkeypox, MedTwitter]"
3,sanidadgob,Ministerio de Sanidad,2023,2,22,es,"Madrid, España",💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,2,2,2,0,2613.0,699628,695,11341,2023-02-22 11:25:00+00:00,1628355168587726851,"[ViruelaDelMono, Monkeypox]"
4,redaccionmedica,Redacción Médica,2023,2,22,es,España,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,0,0,2,0,545.0,97401,7730,64188,2023-02-22 10:37:42+00:00,1628343266893602817,[monkeypox]


In [254]:
df.iloc[1]['raw_content']

"L'@OMSCentrafrique appuie la formation de 4️⃣8️⃣ formateurs et agents de #santé des districts à risques sur la variole du singe (#Monkeypox) intégrée au briefing sur l'épidémie de #choléra, sous l'égide du @MSPCentrafrique. En 2022, la #RCA 🇨🇫 a confirmé 9 cas de #Monkeypox. https://t.co/ZACSyt5rXd"

In [255]:
df.iloc[1]['rendered_content']

"L'@OMSCentrafrique appuie la formation de 4️⃣8️⃣ formateurs et agents de #santé des districts à risques sur la variole du singe (#Monkeypox) intégrée au briefing sur l'épidémie de #choléra, sous l'égide du @MSPCentrafrique. En 2022, la #RCA 🇨🇫 a confirmé 9 cas de #Monkeypox. https://t.co/ZACSyt5rXd"